In [ ]:
from google.colab import drive #mounting drive to read and write data to drive
drive.mount("/content/drive")

In [ ]:
#if running in Jupyter first we install scikit-surprise by going to anaconda comand prompt and typing the command below
#conda install -c conda-forge scikit-surprise
! pip install scikit-surprise
import surprise
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import os
print(os.listdir("/content/drive/MyDrive/books"))

In [ ]:
raw=pd.read_csv('/content/drive/MyDrive/books/ratings.csv')
raw.drop_duplicates(inplace=True)
raw=raw.head(1000)
print('we have',raw.shape[0], 'ratings')
print('the number of unique users we have is:', len(raw.user_id.unique()))
print('the number of unique books we have is:', len(raw.book_id.unique()))
print("The median user rated %d books."%raw.user_id.value_counts().median())
print('The max rating is: %d'%raw.rating.max(),"the min rating is: %d"%raw.rating.min())
raw.head()

In [ ]:
ratings.describe

In [ ]:
rawTrain,rawholdout = train_test_split(ratings, test_size=0.25 )
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(1,5)) 
#into surprise:
data = surprise.Dataset.load_from_df(rawTrain,reader)
holdout = surprise.Dataset.load_from_df(rawholdout,reader)
holdout

In [ ]:
kSplit = surprise.model_selection.split.KFold(n_splits=10, shuffle=True) # split data into folds.

In [ ]:
sim_options = sim_options = {'name': 'cosine','user_based': False  # compute  similarities between items
               } 
collabKNN = surprise.KNNBasic(k=40,sim_options=sim_options) #try removing sim_options. You'll find memory errors. 
rmseKNN = []
rmseSVD = []
rmseCo = []
rmseSlope = []
for trainset, testset in kSplit.split(data): #iterate through the folds.
    collabKNN.fit(trainset)
    predictionsKNN = collabKNN.test(testset)
    rmseKNN.append(surprise.accuracy.rmse(predictionsKNN,verbose=True))#get root means squared error

In [ ]:
funkSVD = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=30,n_epochs=10,biased=True)

In [ ]:
min_error = 1
for trainset, testset in kSplit.split(data): #iterate through the folds.
    funkSVD.fit(trainset)
    predictionsSVD = funkSVD.test(testset)
    rmseSVD.append(surprise.accuracy.rmse(predictionsSVD,verbose=True))#get root means squared error

In [ ]:
coClus = surprise.prediction_algorithms.co_clustering.CoClustering(n_cltr_u=4,n_cltr_i=4,n_epochs=25) 
for trainset, testset in kSplit.split(data): #iterate through the folds.
    coClus.fit(trainset)
    predictionsCoClus = coClus.test(testset)
    rmseCo.append(surprise.accuracy.rmse(predictionsCoClus,verbose=True))#get root means squared error

In [ ]:
slopeOne = surprise.prediction_algorithms.slope_one.SlopeOne()

In [ ]:
for trainset, testset in kSplit.split(data): #iterate through the folds.
    slopeOne.fit(trainset)
    predictionsSlope = slopeOne.test(testset)
    rmseSlope.append(surprise.accuracy.rmse(predictionsSlope,verbose=True))#get root means squared error

In [ ]:
from surprise import AlgoBase

class HybridModel(AlgoBase):

    def __init__(self, models, weights, sim_options={}):
        AlgoBase.__init__(self)
        self.models = models
        self.weights = weights

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        for model in self.models:
            model.fit(trainset)
                
        return self

    def estimate(self, user_id, item_id):
        
        scores_sum = 0
        weights_sum = 0
        
        for i in range(len(self.models)):
            scores_sum += self.models[i].estimate(user_id, item_id) * self.weights[i] # 3*1/4+4*3/4 laga ra
            weights_sum += self.weights[i] # always becomes one
            
        return scores_sum / weights_sum

In [ ]:
class HybridFacto(surprise.AlgoBase):
    def __init__(self,epochs, learning_rate,num_models):
        self.alpha = np.array([1/len(num_models)]*len(num_models))
        self.epochs = epochs
        self.learning_rate = learning_rate
    def fit(self,holdout):
        holdout=holdout.build_full_trainset().build_testset()
        for epoch in range(self.epochs):
          predictions = np.array([collabKNN.test(holdout),funkSVD.test(holdout),coClus.test(holdout),slopeOne.test(holdout)])
          maeGradient = [surprise.accuracy.mae(prediction) for prediction in predictions]
          newalpha = self.alpha - learning_rate * maeGradient  
            #convergence check:
          if newalpha - self.alpha < 0.001:
                break
          self.alpha = newalpha
    def estimate(self,u,i): 
          if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')
            algoResults = np.array([collabKNN.predict(u,i),funkSVD.predict(u,i),coClus.predict(u,i),slopeOne.predict(u,i)])
            return np.sum(np.dot(self.alpha,algoResults))
        

In [ ]:
hybrid = HybridFacto(10,0.05,4)
hybrid.fit(holdout)
rmseHyb = []
for trainset, testset in kSplit.split(data): #iterate through the folds.
    predhybrid = hybrid.test(testset)
    rmseHyb.append(surprise.accuracy.rmse(predhybrid))